In [ ]:
# Google Colabでドライブのデータを使う
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# --- 組み込みモジュール ---
import sys
import pickle
import importlib  # importlibはsysと並べる
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar1 #画像データセット
from tensorflow.keras.layers import Dense, LayerNormalization, Dropout, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

# --- モジュールのパスを追加 ---
sys.path.extend([
    '/content/drive/Shareddrives/MuraolabDocument/技術/機械学習/Transformer/',
    '/content/drive/Shareddrives/MuraolabDocument/技術/機械学習/ViT(Vision Transformer)/'
])

# --- ローカルモジュールのインポート ---
import Transformer_Encoder
import Vision_Transformer as vit

# --- モジュールのリロード（変更を反映） ---
importlib.reload(Transformer_Encoder)
importlib.reload(vit)

In [ ]:
# --- CIFAR-10データの読み込みと前処理 ---
# CIFAR-10は32x32ピクセル、3チャネルの画像データ（10クラス）
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
# ピクセル値を0～1の範囲に正規化
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32') / 255.0

In [ ]:
# --- Vision Transformerモデルの構築 ---
# CIFAR-10は32x32の画像なので、image_heightとimage_widthを32に設定。
# patch_sizeは例えば8に設定すると、画像は4x4のパッチ（合計16個）に分割される。
vit_model = vit.build_vit_model(
    image_height=32,      # 画像の高さ
    image_width=32,       # 画像の幅
    patch_size=8,         # 8x8のパッチ
    num_classes=10,       # CIFAR-10は10クラス
    d_model=64,           # 軽量なモデル例としての埋め込み次元
    num_heads=4,
    ff_dim=256,
    num_layers=4,         # Transformer Encoderブロックの層数
    dropout_rate=0.1,
    l2_lambda=1e-4,
    layer_norm_epsilon=1e-6
)

vit_model.summary()

In [ ]:
# --- モデルのコンパイル ---
vit_model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='sparse_categorical_crossentropy',  # ラベルは整数のためsparse形式を使用
    metrics=['accuracy']
)

# --- モデルの学習 ---
history = vit_model.fit(
    x_train, y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=100
)

In [ ]:
# --- モデルの評価 ---
test_loss, test_acc = vit_model.evaluate(x_test, y_test, batch_size=64)
print(f"Test accuracy: {test_acc:.4f}")